# **Extract GridMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: 

Purpose: This script extracts GridMET data from Google Earth Engine using the Earth Engine API in Python.

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=eI_CYChtbxhoPVvJUyRaWhH6r2_Iqvsl5uav6oiZErs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjVDmydZFu-2toHIFg9BLByr57-9H11ZqtWgCj-3jnSq2PWO61qsQA

Successfully saved authorization token.


In [9]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile

In [2]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
2021_clip/      CDL_subsets/           irrig_lbrb/  lcmap_files/
Boise_CDL/      diversion_timeseries/  IrrMapper/   masked/
CDL_reproject/  irrigation_companies/  LBRB_shp/    subset_test_shp/


In [ ]:

# Algorithm:
# 1. Import shapefile of chosing
# 3. Create a new feature with selected watershed
# 5. Union the watersheds (if necessary), export the watershed(s)
# 6. Get the CDL data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#




In [26]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = gpd.read_file('subset_test_shp/subset.shp')
extent = gpd.GeoSeries(shp_file['geometry'][0]) #get the geometry from shapefile
coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
coords = ee.Geometry.Polygon(coords, proj='EPSG:8826')




epsg:8826


In [41]:
## -------------------------- ## 
## 2. Import the GridMET data ##
## -------------------------- ## 

dataset = ee.Image("IDAHO_EPSCOR/GRIDMET")

<bound method ApiFunction.importApi.<locals>.MakeBoundFunction.<locals>.<lambda> of <ee.image.Image object at 0x7fa5a9f63b90>>